In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "your_password_here"
host = "nv-desktop-services.apporto.com"
port = 30976
db_name = "AAC"
collection_name = "animals"
shelter = AnimalShelter(username, password, host, port, db_name, collection_name)

# Connect to database via CRUD Module
# db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))
df.drop(columns=['_id'], inplace=True)


# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
def encode_image(image_file):
    with open(image_file, 'rb') as f:
        return base64.b64encode(f.read()).decode('ascii')
    
encoded_image = encode_image('logo.png')

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)


app.layout = html.Div([
    html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'width': '10%', 'float': 'right'}),
    html.Center(html.B(html.H1('Justin Osman CS-340 Dashboard'))),
    html.Hr(),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'reset'}
        ],
        value='reset',
        labelStyle={'display': 'inline-block'}
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         row_selectable="single",
                         filter_action="native",
                         sort_action="native",
                         page_size=10,
                         selected_rows=[0],
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'water':
        breeds = ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']
        dff = df[df['breed'].isin(breeds) & (df['age_upon_outcome_in_weeks'] >= 26) & (df['age_upon_outcome_in_weeks'] <= 156)] 
    elif filter_type == 'mountain':
        breeds = ['German Shepard', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']
        dff =df[df['breed'].isin(breeds) & (df['age_upon_outcome_in_weeks'] >= 26) & (df['age_upon_outcome_in_weeks'] <= 156)]
    elif filter_type == 'disaster':
        breeds = ['Doberman Pinscher', 'German Shepard', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
        dff = df[df['breed'].isin(breeds) & (df['age_upon_outcome_in_weeks'] >= 20) & (df['age_upon_outcome_in_weeks'] <= 300)]
    else: 
        dff = df
    return dff.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    if 'breed' in dff.columns:
        fig = px.pie(dff, names='breed', title='Preferred Animals')
    else:
        fig = px.pie()
    return [dcc.Graph(figure=fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    if 'location_lat' in dff.columns and 'location_long' in dff.columns:
        position = [dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']]
    else:
        position = [30.75, -97.48]
        
    tooltip = dff.iloc[row]['breed'] if 'breed' in dff.columns else "Unknown"
    popup_name = dff.iloc[row]['name'] if 'name' in dff.columns else "Unknown"
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=position, children=[
                dl.Tooltip(tooltip),
                dl.Popup([html.H1("Animal Name"), html.P(popup_name)])
            ])
        ])
    ]



app.run_server(mode='inline')


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: 'NoneType' object is not iterable

